In [2]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import lightgbm as lgb
import catboost as cb

from tsfresh.examples import load_robot_execution_failures
from tsfresh import extract_features, select_features
import optuna

from common import EP
from dfdb import DFDB

import types
import copy

Using TensorFlow backend.


In [3]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [8]:
df_train = pd.read_pickle('../feats/df_train.pkl')
df_test = pd.read_pickle('../feats/df_test.pkl')

In [9]:
df_train['label'] = df_train['y'].apply(lambda x:  int(x) if x<15 else 15)
group = df_train['season'].values
group[np.where(group==17)[0]] = 1
df_train['group'] = group
df_train = df_train.drop(columns=['season'])

In [10]:
test_X = df_train.drop(columns=['y','index','group','label']).copy()
test_X.index = df_train['index']
test_y = df_train['y'].copy()
test_y.index = df_train['index']
tsfresh_columns = select_features(test_X, test_y).columns.tolist()

In [11]:
original_columns = df_train.columns.drop(['index','y','label','group']).tolist()

In [4]:
db = DFDB('../trial2/catboost.pkl', auto_commit=False)

In [5]:
df_trial = db.select()

In [6]:
df_trial.loc[[399,448,625,649]][['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
399,2019-05-19 16:16:28.165523,tune 239,18,2.011196,0.000486,2.094451,0.002037,0.083255
448,2019-05-19 21:26:25.175250,remodel 399 use stratified,18,2.030253,0.000001,2.035186,0.000004,0.004932
625,2019-05-19 18:43:45.614456,tune 239 by stratified,18,1.870594,0.000004,1.917066,0.000089,0.046472
649,2019-05-19 21:29:50.830576,remodel 625 use group,18,1.812501,0.000577,2.080786,0.004550,0.268285


In [14]:
cb.CatBoostRegressor?

In [12]:
columns = df_trial.loc[625]['param']['columns']
mytrial = []
selected_columns = []
cv_score = 9999
for col in columns:
    args = copy.deepcopy(df_trial.loc[625]['param'])
    args['columns'] = selected_columns + [col]
    args['scaler']['init'] = {}
    args['algorithm']['fit'] = {}
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='revert RFE')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    if val_mae_mean < cv_score:
        selected_columns.append(col)
        cv_score = val_mae_mean

0:	learn: 6.5120415	total: 69.4ms	remaining: 15.8s
1:	learn: 6.2942932	total: 84.5ms	remaining: 9.59s
2:	learn: 6.0870232	total: 124ms	remaining: 9.32s
3:	learn: 5.8898237	total: 137ms	remaining: 7.7s
4:	learn: 5.7019894	total: 149ms	remaining: 6.68s
5:	learn: 5.5231319	total: 165ms	remaining: 6.13s
6:	learn: 5.3533112	total: 178ms	remaining: 5.63s
7:	learn: 5.1922814	total: 188ms	remaining: 5.2s
8:	learn: 5.0393803	total: 198ms	remaining: 4.85s
9:	learn: 4.8942499	total: 210ms	remaining: 4.6s
10:	learn: 4.7565291	total: 221ms	remaining: 4.38s
11:	learn: 4.6262286	total: 233ms	remaining: 4.22s
12:	learn: 4.5027951	total: 244ms	remaining: 4.06s
13:	learn: 4.3861551	total: 258ms	remaining: 3.96s
14:	learn: 4.2760699	total: 270ms	remaining: 3.85s
15:	learn: 4.1722545	total: 282ms	remaining: 3.75s
16:	learn: 4.0740152	total: 294ms	remaining: 3.67s
17:	learn: 3.9815261	total: 305ms	remaining: 3.58s
18:	learn: 3.8942546	total: 317ms	remaining: 3.5s
19:	learn: 3.8123069	total: 329ms	remaining

161:	learn: 2.6785084	total: 1.94s	remaining: 801ms
162:	learn: 2.6784862	total: 1.95s	remaining: 790ms
163:	learn: 2.6784558	total: 1.96s	remaining: 778ms
164:	learn: 2.6784471	total: 1.98s	remaining: 766ms
165:	learn: 2.6784458	total: 1.98s	remaining: 752ms
166:	learn: 2.6784206	total: 1.99s	remaining: 739ms
167:	learn: 2.6783976	total: 2s	remaining: 727ms
168:	learn: 2.6783906	total: 2.01s	remaining: 715ms
169:	learn: 2.6783896	total: 2.02s	remaining: 700ms
170:	learn: 2.6783869	total: 2.02s	remaining: 686ms
171:	learn: 2.6783860	total: 2.03s	remaining: 673ms
172:	learn: 2.6783842	total: 2.04s	remaining: 659ms
173:	learn: 2.6783828	total: 2.04s	remaining: 646ms
174:	learn: 2.6783816	total: 2.05s	remaining: 634ms
175:	learn: 2.6783812	total: 2.06s	remaining: 621ms
176:	learn: 2.6783770	total: 2.07s	remaining: 608ms
177:	learn: 2.6783763	total: 2.07s	remaining: 594ms
178:	learn: 2.6783754	total: 2.08s	remaining: 581ms
179:	learn: 2.6783563	total: 2.09s	remaining: 568ms
180:	learn: 2.6

92:	learn: 2.6824102	total: 1.14s	remaining: 1.66s
93:	learn: 2.6821376	total: 1.14s	remaining: 1.64s
94:	learn: 2.6818282	total: 1.16s	remaining: 1.63s
95:	learn: 2.6816223	total: 1.16s	remaining: 1.61s
96:	learn: 2.6814070	total: 1.17s	remaining: 1.59s
97:	learn: 2.6812091	total: 1.18s	remaining: 1.58s
98:	learn: 2.6810168	total: 1.19s	remaining: 1.57s
99:	learn: 2.6808339	total: 1.21s	remaining: 1.55s
100:	learn: 2.6806477	total: 1.22s	remaining: 1.54s
101:	learn: 2.6805040	total: 1.23s	remaining: 1.53s
102:	learn: 2.6803644	total: 1.24s	remaining: 1.51s
103:	learn: 2.6802297	total: 1.24s	remaining: 1.49s
104:	learn: 2.6800820	total: 1.26s	remaining: 1.48s
105:	learn: 2.6799540	total: 1.27s	remaining: 1.47s
106:	learn: 2.6798409	total: 1.27s	remaining: 1.45s
107:	learn: 2.6797093	total: 1.29s	remaining: 1.44s
108:	learn: 2.6795555	total: 1.3s	remaining: 1.43s
109:	learn: 2.6794606	total: 1.31s	remaining: 1.42s
110:	learn: 2.6793954	total: 1.32s	remaining: 1.4s
111:	learn: 2.6793128	

22:	learn: 3.5891789	total: 270ms	remaining: 2.42s
23:	learn: 3.5251532	total: 282ms	remaining: 2.4s
24:	learn: 3.4651143	total: 294ms	remaining: 2.4s
25:	learn: 3.4089545	total: 306ms	remaining: 2.39s
26:	learn: 3.3572185	total: 315ms	remaining: 2.36s
27:	learn: 3.3081273	total: 326ms	remaining: 2.34s
28:	learn: 3.2620874	total: 338ms	remaining: 2.33s
29:	learn: 3.2194671	total: 351ms	remaining: 2.33s
30:	learn: 3.1795599	total: 363ms	remaining: 2.32s
31:	learn: 3.1424556	total: 374ms	remaining: 2.3s
32:	learn: 3.1075767	total: 386ms	remaining: 2.29s
33:	learn: 3.0751451	total: 401ms	remaining: 2.3s
34:	learn: 3.0450893	total: 413ms	remaining: 2.29s
35:	learn: 3.0171770	total: 425ms	remaining: 2.28s
36:	learn: 2.9910351	total: 437ms	remaining: 2.27s
37:	learn: 2.9668402	total: 449ms	remaining: 2.26s
38:	learn: 2.9444584	total: 462ms	remaining: 2.25s
39:	learn: 2.9235976	total: 474ms	remaining: 2.24s
40:	learn: 2.9044611	total: 481ms	remaining: 2.21s
41:	learn: 2.8867143	total: 493ms	r

182:	learn: 2.6675367	total: 2.06s	remaining: 518ms
183:	learn: 2.6675357	total: 2.06s	remaining: 505ms
184:	learn: 2.6675351	total: 2.07s	remaining: 493ms
185:	learn: 2.6675032	total: 2.08s	remaining: 481ms
186:	learn: 2.6675026	total: 2.09s	remaining: 469ms
187:	learn: 2.6675016	total: 2.1s	remaining: 457ms
188:	learn: 2.6675014	total: 2.1s	remaining: 445ms
189:	learn: 2.6675008	total: 2.11s	remaining: 433ms
190:	learn: 2.6674817	total: 2.12s	remaining: 422ms
191:	learn: 2.6674700	total: 2.14s	remaining: 412ms
192:	learn: 2.6674695	total: 2.14s	remaining: 400ms
193:	learn: 2.6674687	total: 2.15s	remaining: 388ms
194:	learn: 2.6674667	total: 2.16s	remaining: 377ms
195:	learn: 2.6674638	total: 2.17s	remaining: 365ms
196:	learn: 2.6674580	total: 2.17s	remaining: 353ms
197:	learn: 2.6674559	total: 2.19s	remaining: 342ms
198:	learn: 2.6674512	total: 2.19s	remaining: 331ms
199:	learn: 2.6674425	total: 2.21s	remaining: 320ms
200:	learn: 2.6674425	total: 2.21s	remaining: 308ms
201:	learn: 2.

...

4:	learn: 5.6983728	total: 141ms	remaining: 6.3s
5:	learn: 5.5189478	total: 167ms	remaining: 6.21s
6:	learn: 5.3490123	total: 195ms	remaining: 6.18s
7:	learn: 5.1860050	total: 217ms	remaining: 6s
8:	learn: 5.0314365	total: 246ms	remaining: 6s
9:	learn: 4.8848036	total: 270ms	remaining: 5.92s
10:	learn: 4.7449568	total: 297ms	remaining: 5.88s
11:	learn: 4.6130357	total: 324ms	remaining: 5.86s
12:	learn: 4.4884118	total: 350ms	remaining: 5.81s
13:	learn: 4.3706678	total: 379ms	remaining: 5.81s
14:	learn: 4.2573238	total: 404ms	remaining: 5.77s
15:	learn: 4.1510841	total: 432ms	remaining: 5.75s
16:	learn: 4.0518371	total: 455ms	remaining: 5.67s
17:	learn: 3.9579518	total: 479ms	remaining: 5.62s
18:	learn: 3.8677156	total: 505ms	remaining: 5.59s
19:	learn: 3.7819283	total: 526ms	remaining: 5.5s
20:	learn: 3.7016659	total: 554ms	remaining: 5.48s
21:	learn: 3.6263416	total: 582ms	remaining: 5.47s
22:	learn: 3.5550130	total: 611ms	remaining: 5.47s
23:	learn: 3.4883099	total: 640ms	remaining: 

165:	learn: 2.4193929	total: 4.46s	remaining: 1.69s
166:	learn: 2.4186556	total: 4.49s	remaining: 1.67s
167:	learn: 2.4175096	total: 4.51s	remaining: 1.64s
168:	learn: 2.4166495	total: 4.54s	remaining: 1.61s
169:	learn: 2.4153972	total: 4.56s	remaining: 1.58s
170:	learn: 2.4150318	total: 4.59s	remaining: 1.56s
171:	learn: 2.4149548	total: 4.62s	remaining: 1.53s
172:	learn: 2.4144139	total: 4.65s	remaining: 1.5s
173:	learn: 2.4132295	total: 4.68s	remaining: 1.48s
174:	learn: 2.4126652	total: 4.7s	remaining: 1.45s
175:	learn: 2.4120429	total: 4.73s	remaining: 1.42s
176:	learn: 2.4119963	total: 4.75s	remaining: 1.4s
177:	learn: 2.4116300	total: 4.78s	remaining: 1.37s
178:	learn: 2.4111781	total: 4.81s	remaining: 1.34s
179:	learn: 2.4105346	total: 4.83s	remaining: 1.31s
180:	learn: 2.4101263	total: 4.86s	remaining: 1.29s
181:	learn: 2.4095505	total: 4.89s	remaining: 1.26s
182:	learn: 2.4090107	total: 4.92s	remaining: 1.24s
183:	learn: 2.4085620	total: 4.94s	remaining: 1.21s
184:	learn: 2.4

97:	learn: 2.4775898	total: 2.54s	remaining: 3.4s
98:	learn: 2.4761081	total: 2.57s	remaining: 3.38s
99:	learn: 2.4747288	total: 2.6s	remaining: 3.35s
100:	learn: 2.4738727	total: 2.62s	remaining: 3.32s
101:	learn: 2.4727706	total: 2.65s	remaining: 3.3s
102:	learn: 2.4717149	total: 2.67s	remaining: 3.27s
103:	learn: 2.4711144	total: 2.72s	remaining: 3.27s
104:	learn: 2.4703851	total: 2.74s	remaining: 3.24s
105:	learn: 2.4698852	total: 2.77s	remaining: 3.21s
106:	learn: 2.4679430	total: 2.8s	remaining: 3.19s
107:	learn: 2.4665380	total: 2.82s	remaining: 3.16s
108:	learn: 2.4658926	total: 2.85s	remaining: 3.13s
109:	learn: 2.4649983	total: 2.87s	remaining: 3.1s
110:	learn: 2.4642670	total: 2.9s	remaining: 3.08s
111:	learn: 2.4637251	total: 2.93s	remaining: 3.06s
112:	learn: 2.4625246	total: 2.95s	remaining: 3.03s
113:	learn: 2.4616107	total: 2.98s	remaining: 3.01s
114:	learn: 2.4609130	total: 3.01s	remaining: 2.98s
115:	learn: 2.4594237	total: 3.03s	remaining: 2.96s
116:	learn: 2.4585382

27:	learn: 3.2547035	total: 746ms	remaining: 5.36s
28:	learn: 3.2054665	total: 771ms	remaining: 5.31s
29:	learn: 3.1597178	total: 796ms	remaining: 5.28s
30:	learn: 3.1175168	total: 824ms	remaining: 5.26s
31:	learn: 3.0773433	total: 842ms	remaining: 5.18s
32:	learn: 3.0388669	total: 873ms	remaining: 5.18s
33:	learn: 3.0041452	total: 897ms	remaining: 5.14s
34:	learn: 2.9697052	total: 925ms	remaining: 5.13s
35:	learn: 2.9398697	total: 955ms	remaining: 5.12s
36:	learn: 2.9100410	total: 982ms	remaining: 5.1s
37:	learn: 2.8828644	total: 1.01s	remaining: 5.08s
38:	learn: 2.8579218	total: 1.04s	remaining: 5.05s
39:	learn: 2.8343464	total: 1.06s	remaining: 5.03s
40:	learn: 2.8127001	total: 1.09s	remaining: 5s
41:	learn: 2.7913754	total: 1.12s	remaining: 5s
42:	learn: 2.7722629	total: 1.15s	remaining: 4.97s
43:	learn: 2.7539031	total: 1.17s	remaining: 4.94s
44:	learn: 2.7363818	total: 1.2s	remaining: 4.92s
45:	learn: 2.7208429	total: 1.23s	remaining: 4.89s
46:	learn: 2.7058729	total: 1.26s	remai

187:	learn: 2.4044438	total: 5.03s	remaining: 1.1s
188:	learn: 2.4035859	total: 5.06s	remaining: 1.07s
189:	learn: 2.4030834	total: 5.08s	remaining: 1.04s
190:	learn: 2.4028849	total: 5.09s	remaining: 1.01s
191:	learn: 2.4028388	total: 5.11s	remaining: 984ms
192:	learn: 2.4021501	total: 5.13s	remaining: 958ms
193:	learn: 2.4012452	total: 5.16s	remaining: 931ms
194:	learn: 2.4005657	total: 5.19s	remaining: 905ms
195:	learn: 2.4005588	total: 5.2s	remaining: 875ms
196:	learn: 2.3994022	total: 5.23s	remaining: 849ms
197:	learn: 2.3988040	total: 5.25s	remaining: 822ms
198:	learn: 2.3982765	total: 5.28s	remaining: 796ms
199:	learn: 2.3982633	total: 5.29s	remaining: 767ms
200:	learn: 2.3982277	total: 5.3s	remaining: 739ms
201:	learn: 2.3976093	total: 5.33s	remaining: 712ms
202:	learn: 2.3971466	total: 5.35s	remaining: 686ms
203:	learn: 2.3966795	total: 5.38s	remaining: 659ms
204:	learn: 2.3966610	total: 5.41s	remaining: 633ms
205:	learn: 2.3961224	total: 5.43s	remaining: 607ms
206:	learn: 2.3

In [13]:
len(selected_columns)

14

In [8]:
#check feature_importances
# df_feature_importances = df_trial.loc[235]['df_feature_importances']
# if type(df_feature_importances)==pd.DataFrame:
#     sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
# else:
#     sorted_columns = df_trial.loc[235]['param']['columns']
# df_feature_importances.sort_values(by=['average_permutation_weight'], ascending=False)
# len(sorted_columns)

In [11]:
# param = df_trial.loc[342]['param'].copy()
param = {'columns': tsfresh_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 589,
   'depth': 6,
   'learning_rate': 0.05293979792364842,
   'l2_leaf_reg': 78.065140245968,
   'bagging_temperature': 0.9302786271852079,
   'random_strength': 0.4247048326178351,
   'random_state': 651},
  'fit': {}},
}

In [1]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=20, key='average_model_weight', remark='group3 RFE')
for trial_i in mytrial:
    db.insert(trial_i)

In [15]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [16]:
df_trial[['datetime', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
0,2019-05-17 19:10:11.182668,1071,1.601879,0.000069,2.121013,0.007367,0.519134
1,2019-05-18 04:01:18.357701,1051,1.598117,0.000113,2.122386,0.007074,0.524270
2,2019-05-18 12:29:57.440833,1031,1.599668,0.000188,2.117059,0.007864,0.517391
3,2019-05-18 20:49:18.314789,1011,1.597300,0.000085,2.120985,0.007927,0.523685


In [36]:
top4_sorted_columns = []
for i in range(4):
    df_feature_importances = df_trial.loc[i]['df_feature_importances']
    sorted_columns = EP.evaluate(df_feature_importances, key='average_model_weight')
    top4_sorted_columns += sorted_columns[:100]

In [39]:
sorted_columns = list(set(top4_sorted_columns))
len(top4_sorted_columns), len(set(top4_sorted_columns))

(400, 134)

In [40]:
# param = df_trial.loc[342]['param'].copy()
param = {'columns': sorted_columns,
 'kfold': {'n_splits': 3,
  'random_state': 1985,
  'shuffle': True,
  'type': 'group'},
 'scaler': {'cls': 'StandardScaler'},
 'algorithm': {'cls': 'cb.CatBoostRegressor',
  'init': {'num_trees': 589,
   'depth': 6,
   'learning_rate': 0.05293979792364842,
   'l2_leaf_reg': 78.065140245968,
   'bagging_temperature': 0.9302786271852079,
   'random_strength': 0.4247048326178351,
   'random_state': 651},
  'fit': {}},
}

In [2]:
mytrial =[]
EP.select_features_(df_train, param, mytrial, nfeats_best=10, nfeats_removed_per_try=20, key='average_model_weight', remark='group3 RFE2')
for trial_i in mytrial:
    db.insert(trial_i)

In [46]:
df_trial = db.select()
df_trial[df_trial['remark']=='group3 RFE2'][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
4,2019-05-18 22:57:34.706729,134,1.631104,0.000029,2.107990,0.004890,0.476886
5,2019-05-18 23:04:57.264779,114,1.628535,0.000011,2.107337,0.005743,0.478802
6,2019-05-18 23:10:27.350303,94,1.637958,0.000004,2.104357,0.005169,0.466399
7,2019-05-18 23:14:22.274333,74,1.643851,0.000050,2.111248,0.005386,0.467397
8,2019-05-18 23:16:55.341345,54,1.662980,0.000098,2.101211,0.003817,0.438231
9,2019-05-18 23:18:34.737478,34,1.700759,0.000115,2.098778,0.004548,0.398019
10,2019-05-18 23:19:36.014654,14,1.791819,0.000385,2.093661,0.004730,0.301842


In [3]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[9]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 9')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

In [80]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [85]:
df_trial[(df_trial['remark']=='tune 9')&(df_trial['mae_diff']<.1)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
175,2019-05-19 13:16:11.593624,34,2.001394,0.000772,2.098855,0.002856,0.097461
133,2019-05-19 12:48:16.282690,34,2.012604,0.000851,2.103320,0.004457,0.090716
184,2019-05-19 13:19:23.692973,34,2.025755,0.001135,2.115385,0.003147,0.089630
205,2019-05-19 13:27:37.605488,34,2.031798,0.001240,2.120268,0.003316,0.088470


In [86]:
db.commit()

In [4]:
mytrial =[]
EP.select_features_(df_train, df_trial.loc[225]['param'], mytrial, nfeats_best=10, nfeats_removed_per_try=1, key='average_model_weight', remark='group3 RFE3')
for trial_i in mytrial:
    db.insert(trial_i)

In [107]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [108]:
df_trial[(df_trial['remark']=='group3 RFE3')][['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
211,2019-05-19 14:01:13.362562,34,2.012592,0.000867,2.104591,0.004387,0.091999
212,2019-05-19 14:02:10.587266,33,2.011891,0.000960,2.103509,0.004173,0.091618
213,2019-05-19 14:03:04.502816,32,2.011686,0.000879,2.105717,0.004033,0.094031
214,2019-05-19 14:03:56.238684,31,2.011478,0.000974,2.103843,0.004536,0.092365
215,2019-05-19 14:04:46.032615,30,2.011883,0.000935,2.103560,0.004511,0.091676
216,2019-05-19 14:05:35.946079,29,2.011029,0.000916,2.102193,0.004018,0.091165
217,2019-05-19 14:06:22.262427,28,2.012179,0.000963,2.103864,0.003936,0.091685
218,2019-05-19 14:07:06.267310,27,2.010659,0.000878,2.104995,0.004260,0.094336
219,2019-05-19 14:07:48.441406,26,2.010527,0.000957,2.104252,0.004290,0.093725
220,2019-05-19 14:08:29.891292,25,2.011318,0.000937,2.103217,0.004011,0.091899


In [110]:
cb_columns = ['iqr_6',
 'abs_q01_4',
 'q25_roll_std_100',
 'median__roll_std',
 'q05_roll_std_100',
 'q05_roll_std_10',
 'abs_q75_6',
 'abs_q25_5',
 'q05_roll_std_1000',
 'abs_q01_5',
 "number_peaks{'n': 10}",
 '5000std_quantile05',
 "number_peaks{'n': 3}",
 'abs_q75_7',
 'spkt_welch_density__coeff_3',
 'q75_roll_mean_10',
 '5000quantile75mean_',
 'Hilbert_mean_6']

In [5]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[239]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'group'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 239')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

In [113]:
len(mytrial)

200

In [115]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [120]:
df_trial[(df_trial['remark']=='tune 239')&(df_trial['mae_diff']<.09)].sort_values(by=['val_mae'])[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
399,2019-05-19 16:16:28.165523,18,2.011196,0.000486,2.094451,0.002037,0.083255
418,2019-05-19 16:24:59.264481,18,2.010556,0.000663,2.097445,0.004091,0.086889
442,2019-05-19 16:36:55.296284,18,2.013315,0.000613,2.100960,0.001960,0.087645
352,2019-05-19 15:51:04.411734,18,2.028762,0.000384,2.103298,0.002808,0.074536
396,2019-05-19 16:15:31.916870,18,2.030920,0.000522,2.107848,0.002218,0.076928
405,2019-05-19 16:18:24.352245,18,2.025673,0.000641,2.111320,0.002253,0.085647


In [121]:
db.commit()

In [6]:
param = copy.deepcopy(df_trial.loc[399]['param'])
param['kfold']['type'] = 'stratified'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 399 use stratified')

In [125]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [126]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
448,2019-05-19 21:26:25.175250,18,2.030253,0.000001,2.035186,0.000004,0.004932


In [39]:
db.commit()

In [ ]:
mytrial =[]

#  tune hypterparameters
def objective(trial):
        
    num_trees = trial.suggest_int('num_trees', 200, 1000)
    depth = trial.suggest_int('depth', 2, 10)
    learning_rate = trial.suggest_uniform('learning_rate', 0.01, 0.4)
    l2_leaf_reg = trial.suggest_uniform('l2_leaf_reg', 0.0001, 1)
    bagging_temperature = trial.suggest_uniform('bagging_temperature', .6, 1)
    random_strength = trial.suggest_uniform('random_strength', .001, 1)
    random_state = trial.suggest_int('random_state', 1, 9999)
        
    args={
        'columns':df_trial.loc[239]['param']['columns'],
        'kfold':{
            'n_splits': 3,
            'random_state': 1985,
            'shuffle': True,
            'type': 'stratified'
        },
        'scaler':{
            'cls':'StandardScaler',
        },
        'algorithm':{
            'cls':'cb.CatBoostRegressor',
            'init':{
                "num_trees":num_trees,
                "depth":depth,
                "learning_rate":learning_rate,
                "l2_leaf_reg":l2_leaf_reg,
                "bagging_temperature":bagging_temperature,
                "random_strength":random_strength,
                "random_state":random_state,
            },
            'fit':{
            },
        }
    }
    
    df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, args, df_test = df_test, trial=mytrial, remark='tune 239 by stratified')
    val_mae_mean = np.mean(df_his.valid)
    val_mae_var = np.var(df_his.valid)
    train_mae_mean = np.mean(df_his.train)
    train_mae_var = np.var(df_his.train)
    
    trial.set_user_attr('val_mae', val_mae_mean)
    trial.set_user_attr('train_mae', train_mae_mean)
    trial.set_user_attr('mae_diff', val_mae_mean-train_mae_mean)
    trial.set_user_attr('val_mae_var', val_mae_var)

    return np.abs(val_mae_mean - train_mae_mean)*val_mae_mean

study = optuna.create_study()
study.optimize(objective, n_trials=200)

In [127]:
for trial_i in mytrial:
    db.insert(trial_i)

In [128]:
df_trial = db.select()
df_trial[(df_trial['remark']=='tune 239 by stratified')&(df_trial['mae_diff']<.05)].sort_values(by=['val_mae'])[['datetime','remark', 'nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']].head(10)

,datetime,remark,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
625,2019-05-19 18:43:45.614456,tune 239 by stratified,18,1.870594,3.552597e-06,1.917066,0.000089,0.046472
501,2019-05-19 17:23:10.885796,tune 239 by stratified,18,1.869067,8.096949e-06,1.917206,0.000139,0.048138
552,2019-05-19 17:59:33.597379,tune 239 by stratified,18,1.869971,3.906740e-05,1.919193,0.000145,0.049222
621,2019-05-19 18:42:00.925931,tune 239 by stratified,18,1.871447,2.573961e-07,1.919232,0.000104,0.047786
531,2019-05-19 17:43:50.287298,tune 239 by stratified,18,1.873281,1.539356e-05,1.919493,0.000131,0.046213
583,2019-05-19 18:18:25.954161,tune 239 by stratified,18,1.874073,1.223199e-06,1.920116,0.000119,0.046043
642,2019-05-19 18:51:48.166331,tune 239 by stratified,18,1.883360,4.913177e-06,1.923299,0.000138,0.039939
478,2019-05-19 17:03:02.267204,tune 239 by stratified,18,1.882801,1.295311e-06,1.925255,0.000092,0.042454
479,2019-05-19 17:04:29.586846,tune 239 by stratified,18,1.883649,7.780033e-06,1.927169,0.000145,0.043520
502,2019-05-19 17:23:48.136932,tune 239 by stratified,18,1.892556,3.577277e-05,1.927895,0.000080,0.035339


In [35]:
db.commit()

In [7]:
param = copy.deepcopy(df_trial.loc[625]['param'])
param['kfold']['type'] = 'group'
# run one try
mytrial=[]
df_his,  df_feature_importances, df_valid_pred, df_test_pred =  EP.process(df_train, param, df_test = df_test, trial=mytrial, remark='remodel 625 use group')

In [130]:
for trial_i in mytrial:
    db.insert(trial_i)
df_trial = db.select()

In [131]:
df_trial.tail(1)[['datetime','nfeatures', 'train_mae','train_mae_var','val_mae','val_mae_var','mae_diff']]

,datetime,nfeatures,train_mae,train_mae_var,val_mae,val_mae_var,mae_diff
649,2019-05-19 21:29:50.830576,18,1.812501,0.000577,2.080786,0.00455,0.268285


In [132]:
db.commit()